In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

In [2]:
sm=pd.read_csv('./stockData/005930.csv', index_col=0, parse_dates=True)
sm=sm.sort_index()
sm=sm.dropna()
sm=sm.drop('volume',axis=1)
sm=sm/10000
 # sm["sma120"]=sm['close'].rolling(window=20).mean()
 # sm['sma240']=sm['close'].rolling(window=40).mean()
sm['next close']=sm['close'].shift(-1)

In [3]:
sm.loc[sm['next close']>sm['close'], 'close']=1
sm=sm.sample(frac=1)
sm=sm.dropna()

inputX = np.array(sm.iloc[1500:,0:4])
OutputY=np.array(sm.loc[:, 'next close']).reshape(-1,1)


In [28]:
X_test=np.array(sm.iloc[-1:, 0:3])

Input=4
Wei=6
Output=1

tf.reset_default_graph()

x=tf.placeholder(tf.float32, shape=[None,Input],name="x")
y=tf.placeholder(tf.float32, shape=[None,Output],name='y')

W1=tf.Variable(tf.truncated_normal([Input,Wei]),dtype=tf.float32,name='W1')
Bi=tf.Variable(tf.zeros([Wei]),dtype=tf.float32,name='Bi')
H1=tf.nn.relu(tf.add(tf.matmul(x,W1), Bi, name= 'H1'))

Wo=tf.Variable(tf.truncated_normal([Wei,Output]),dtype=tf.float32,name='Wo')
Bo=tf.Variable(tf.zeros([Output]),dtype=tf.float32, name='Bo')


nY=tf.nn.relu(tf.matmul(H1,Wo) + Bo, name='nY')
cost=tf.reduce_mean(tf.square(tf.subtract(nY,y)))

alpha=tf.placeholder(tf.float32, shape=(),name='alpha')
optimizer=tf.train.AdamOptimizer(alpha)
train=optimizer.minimize(cost)


In [45]:
nBatchCnt=5
nBatchSize=int(inputX.shape[0]/nBatchCnt)

sess=tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(10000):
    for n in range(nBatchCnt):
        nfrom= n * nBatchSize
        nto= n * nBatchSize + nBatchSize
    
        if n == nBatchCnt -1:
            nto= inputX.shape[0]
        
        bx=inputX[nfrom: nto]
        by=OutputY[nfrom : nto]
    
        _, xcost = sess.run([train, cost], feed_dict={x : bx, y : by, alpha: 0.005})
    
    if i % 1000 == 0:
        print('cost = %.6f' % xcost)

cost = 8.280639
cost = 0.914702
cost = 0.912634
cost = 0.911122
cost = 0.910238
cost = 0.909351
cost = 0.908827
cost = 0.908381
cost = 0.908182
cost = 0.907634


In [49]:
yHat=(sess.run(nY, feed_dict={x : inputX}))*10000
print(yHat)
yHat=np.array([1 if x > 0.5 else 0 for x in yHat]).reshape(-1,1)

[[33150.684]
 [33272.082]
 [31915.438]
 [33189.004]
 [32990.957]
 [32042.936]
 [31466.732]
 [33286.902]
 [33439.258]
 [33229.223]
 [31474.328]
 [32715.969]
 [31586.354]
 [33557.984]
 [33429.42 ]
 [31418.021]
 [33065.547]
 [33005.87 ]
 [31814.46 ]
 [33218.793]
 [31497.816]
 [32410.932]
 [31301.684]
 [32320.285]
 [31269.758]
 [33649.758]
 [32681.547]
 [33166.805]
 [33077.66 ]
 [33307.688]
 [33208.05 ]
 [31790.639]
 [33191.105]
 [33309.992]
 [33227.625]
 [31487.648]
 [33657.97 ]
 [33332.613]
 [33489.957]
 [33566.81 ]
 [31882.229]
 [32493.54 ]
 [33316.21 ]
 [33093.676]
 [33308.1  ]
 [32763.51 ]
 [33314.418]
 [33352.633]
 [31619.217]
 [31743.816]
 [32794.266]
 [33031.03 ]
 [33218.08 ]
 [32434.252]
 [33220.97 ]
 [33254.797]
 [31672.549]
 [33176.293]
 [33254.38 ]
 [32479.078]
 [33253.95 ]
 [31900.098]
 [33318.492]
 [32934.402]
 [33754.07 ]
 [33264.918]
 [33723.367]
 [33146.477]
 [33137.246]
 [33148.836]
 [31889.896]
 [32310.87 ]
 [32994.938]
 [33340.113]
 [33237.29 ]
 [31939.062]
 [31972.799]

In [47]:
accuracy = (OutputY == yHat).sum() / len(testX)

C:\Anaconda3.7\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


AttributeError: 'bool' object has no attribute 'sum'